
#  Azure Landing Zone: WordPress Deployment Example (Terraform + LAMP)

## Overview

This example demonstrates how to automate the deployment of a WordPress website using a LAMP stack on an Azure Virtual Machine. We'll use **Terraform** for infrastructure provisioning and a **Bash script** for application setup.

---



## Architecture Summary

- **Virtual Network (VNet)** with a **subnet**
- **Network Interface (NIC)** attached to a **Linux Virtual Machine (VM)**
- **Public IP address** and **Network Security Group (NSG)**
- The VM is configured with **Apache, PHP, MySQL**, and **WordPress**



## Terraform File Structure

```
wordpress-lamp/
├── main.tf            # Main infrastructure config
├── variables.tf       # Input variables
├── outputs.tf         # Outputs like public IP
├── install.sh         # VM bootstrap script to install WordPress
```


In [ ]:

# main.tf (Terraform snippet)

provider "azurerm" {
  features {}
}

resource "azurerm_resource_group" "rg" {
  name     = "rg-wordpress"
  location = "East US"
}

resource "azurerm_virtual_network" "vnet" {
  name                = "vnet-wordpress"
  address_space       = ["10.0.0.0/16"]
  location            = azurerm_resource_group.rg.location
  resource_group_name = azurerm_resource_group.rg.name
}

resource "azurerm_subnet" "subnet" {
  name                 = "subnet-wordpress"
  resource_group_name  = azurerm_resource_group.rg.name
  virtual_network_name = azurerm_virtual_network.vnet.name
  address_prefixes     = ["10.0.1.0/24"]
}



## VM and Network Configuration (Terraform)


In [ ]:

resource "azurerm_network_interface" "nic" {
  name                = "nic-wordpress"
  location            = azurerm_resource_group.rg.location
  resource_group_name = azurerm_resource_group.rg.name

  ip_configuration {
    name                          = "internal"
    subnet_id                     = azurerm_subnet.subnet.id
    private_ip_address_allocation = "Dynamic"
  }
}

resource "azurerm_linux_virtual_machine" "vm" {
  name                = "vm-wordpress"
  resource_group_name = azurerm_resource_group.rg.name
  location            = azurerm_resource_group.rg.location
  size                = "Standard_B1s"
  admin_username      = "azureuser"
  admin_password      = "YourPassword123!"
  network_interface_ids = [azurerm_network_interface.nic.id]

  os_disk {
    caching              = "ReadWrite"
    storage_account_type = "Standard_LRS"
  }

  source_image_reference {
    publisher = "Canonical"
    offer     = "UbuntuServer"
    sku       = "18.04-LTS"
    version   = "latest"
  }

  custom_data = filebase64("install.sh")
}



## 📜 Bash Script: install.sh

This script installs the full LAMP stack and sets up WordPress.



#!/bin/bash

apt-get update
apt-get install -y apache2 php php-mysql mysql-server wget unzip
systemctl start apache2
systemctl enable apache2

cd /var/www/html
wget https://wordpress.org/latest.tar.gz
tar -xzf latest.tar.gz
cp -r wordpress/* .
chown -R www-data:www-data /var/www/html
chmod -R 755 /var/www/html
rm -rf wordpress latest.tar.gz



## Accessing the Website

After applying Terraform, get the public IP address:

```bash
terraform output
```

Then, open a browser and visit:  
**http://<your-public-ip>**

You should see the WordPress setup page 

---

## Summary

This example demonstrates how to:
- Use **Terraform** to provision infrastructure
- Bootstrap a **LAMP stack** with **Bash**
- Automate a full **WordPress** deployment on Azure

> Infrastructure as Code + Automation = Fast, repeatable cloud deployments!
